In [1]:
import os
import numpy as np 
import time
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader 
from chess import pgn 
from tqdm import tqdm 


In [2]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("../data/pgn/") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 28)
games = []
i = 1

for file in tqdm(files):
    games.extend(load_pgn(f"../data/pgn/{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

 34%|███▍      | 27/79 [02:13<04:16,  4.93s/it]


In [3]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 41570


In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.0.0+cu118
True


##CONVIRTIENDO LOS DATOS EN TENSORES 

In [5]:
from Entrada import create_input_for_nn , encode_moves
from Modelo import ChessModel
from Dataset import ChessDataset



In [6]:
# Funciones auxiliares
def process_batch(games_batch):
    X_batch, y_batch = create_input_for_nn(games_batch)
    y_batch, move_to_int = encode_moves(y_batch)
    return torch.tensor(X_batch, dtype=torch.float32), torch.tensor(y_batch, dtype=torch.long), move_to_int

def load_games_in_batches(batch_size, total_games):
    for i in range(0, total_games, batch_size):
        yield games[i:i + batch_size]

In [7]:

batch_size = 3000
total_games = len(games)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

sample_games = games[:batch_size]
_, y_sample, move_to_int = process_batch(sample_games)
num_classes = len(move_to_int)

Using device: cuda


In [9]:
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for games_batch in load_games_in_batches(batch_size, total_games):
        X_batch, y_batch, move_to_int = process_batch(games_batch)
        
        # Create Dataset and DataLoader
        dataset = ChessDataset(X_batch, y_batch)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()

            outputs = model(inputs)  # Raw logits

            # Compute loss
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            running_loss += loss.item()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / total_games:.4f}, Time: {minutes}m{seconds}s')

KeyboardInterrupt: 